# 2D model without preprocessing

In [ ]:
# 3rd-party imports
import numpy as np
import nibabel as nib
import tensorflow as tf
import keras as k
import matplotlib.pyplot as plt

## Load data

In [ ]:
from utils.load_data import load_dataset

train_nib, val_nib = load_dataset('../data', verbose=True)

In [ ]:
train_nib.keys()

In [ ]:
[len(v) for v in train_nib.values()]

In [ ]:
[len(v) for v in val_nib.values()]

In [ ]:
img = train_nib['T1'][0].get_fdata()[:,:,10]

plt.imshow(img)

img.min(), img.max()

## Prepare the model